In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import warnings
warnings.filterwarnings("ignore")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%%capture
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from os import listdir
from PIL import ImageOps, Image
from glob import glob
from sklearn.model_selection import train_test_split as splitTrainingData
from sklearn.metrics import accuracy_score

In [0]:
# Fonction d'entrainement
def training(model, train_dataloader, valid_dataloader=None, epoch=5, learning_rate=0.1, use_gpu=False):
  
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
  
  train_accu = []
  train_losses = []
  
  if valid_dataloader:
    val_accu = []
    val_loss = []
    
  for i in range(epoch):
    model.train()
    
    print('Starting epoch number {} on {} ...'.format(i+1, epoch))
    
    true = []
    pred = []
    train_loss = []
    len_train = len(train_dataloader)
    for ind, batch in enumerate(train_dataloader):
      print('\rBatch : {}/{}'.format(ind+1, len_train), end='')
      inputs, targets = batch
      
      # Aller le GPU, fais ton taff
      if use_gpu:
        inputs = inputs.cuda()
        targets = targets.cuda()
      
      optimizer.zero_grad()
        
      outputs = model(inputs)
      
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()
      
      predictions = outputs.max(dim=1)[1]
      
      train_loss.append(loss.item())
      true.extend(targets.data.cpu().numpy().tolist())
      pred.extend(predictions.data.cpu().numpy().tolist())
    
    accu_score = accuracy_score(true, pred) * 100
    loss_score = sum(train_loss) / len(train_loss)
    
    print('\nTrain score : Accuracy = {:.2f} - Loss = {:.2f}'.format(accu_score, loss_score), end='')
    
    
    train_accu.append(accuracy_score(true, pred) * 100)
    train_losses.append(loss_score)
      
    if valid_dataloader:
      vaccu, vloss = validating(model, valid_dataloader, use_gpu)
      print(' | Validation score : Accuracy = {:.2f} - Loss = {:.2f}'.format(vaccu, vloss))
      val_accu.append(vaccu)
      val_loss.append(vloss)
    else:
      print()
  
  if valid_dataloader:
    return train_accu, train_losses, val_accu, val_loss
  else:
    return train_accu, train_loss
  
  return None, None
  

# Fonction de validation
def validating(model, dataloader, use_gpu=False):
  true =[]
  pred = []
  val_loss = []

  criterion = nn.NLLLoss()
  model.eval()

  for batch in dataloader:

    inputs, targets = batch

    # On envoit les données au GPU pour le traitement
    if use_gpu:
      inputs = inputs.cuda()
      targets = targets.cuda()
    
    outputs = model(inputs)

    predictions = outputs.max(dim=1)[1]

    val_loss.append(criterion(outputs, targets).item())
    true.extend(targets.data.cpu().numpy().tolist())
    pred.extend(predictions.data.cpu().numpy().tolist())
  
  accu_score = accuracy_score(true, pred) * 100
  loss_score = sum(val_loss) / len(val_loss)
  return accu_score, loss_score


# Fonction de prédiction
#def predict(model, sentence, vocab, max_len):
#  tokens = word_tokenize(sentence)
#  X_test_len = len(tokens)
#  X_test = prepareData(tokens, vocab, max_len)
#  X_test = torch.LongTensor(X_test).cuda()
#  X_test_len = torch.FloatTensor(X_test_len).cuda()
#  output = model(X_test, X_test_len)
#  prediction = output.max(dim=1)[1]
#  print(prediction)

In [0]:
%%script false
# Cellule pour éviter les doublons dans les positifs et négatifs.
# Lancer la cellule 2 fois pour que le calcul se fasse

files_pos = [f for f in listdir("/content/gdrive/My Drive/datas/pixabay/dogs/1")]
files_neg = [f for f in listdir("/content/gdrive/My Drive/datas/pixabay/dogs/0")]

cnt = 0
for f in files_pos:
  if f in files_neg:
    cnt += 1
    
print(cnt)

Dans ce cadre précis il faut ajouter du padding à chaque image. Pour se faire, nous devons rechercher la largeur max et la longueur max.

In [0]:
class AddPadding:
  def __init__(self, maxHeight, maxWidth):
    self.maxHeight = maxHeight
    self.maxWidth = maxWidth

  def __call__(self, img):
    # Utilisation du code https://discuss.pytorch.org/t/add-padding-to-images/24309/3
    delta_width = self.maxWidth - img.size[0]
    delta_height = self.maxHeight - img.size[1]
    pad_width = delta_width //2
    pad_height = delta_height //2
    padding = (pad_width,pad_height,delta_width-pad_width,delta_height-pad_height)
    return ImageOps.expand(img, padding)

def create_dataloader(dataset):
    train_loader = DataLoader(
        dataset,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return train_loader

In [6]:
# On charge les dataloaders (en supprimant les images précédentes pour libérer l'espace)
img_path = "/content/gdrive/My Drive/datas/pixabay/dogs/"
maxH = 340
maxW = 1162
all_transforms = transforms.Compose(
    [
     AddPadding(maxH, maxW),
     transforms.ToTensor()
    ]
)
train_dataset = torchvision.datasets.ImageFolder(
    root=img_path,
    transform=all_transforms
)

# Dataloader
data_loader = create_dataloader(train_dataset)

model = torchvision.models.resnet18(pretrained=True)

# La classification se fait sur 2 classes uniquement
model.fc = nn.Linear(512, 2)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [7]:
model.conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [8]:
training(model.cuda(), data_loader, use_gpu=True)

Starting epoch number 1 on 5 ...
Batch : 323/323
Train score : Accuracy = 69.61 - Loss = 0.86
Starting epoch number 2 on 5 ...
Batch : 323/323
Train score : Accuracy = 72.61 - Loss = 0.60
Starting epoch number 3 on 5 ...
Batch : 323/323
Train score : Accuracy = 72.64 - Loss = 0.59
Starting epoch number 4 on 5 ...
Batch : 323/323
Train score : Accuracy = 72.78 - Loss = 0.59
Starting epoch number 5 on 5 ...
Batch : 323/323
Train score : Accuracy = 72.73 - Loss = 0.58


([69.61406227301342,
  72.6066534308266,
  72.63570771391215,
  72.77613674882572,
  72.72771294368312],
 [0.5855599641799927,
  0.5272305011749268,
  0.6003022193908691,
  0.5769608616828918,
  0.5191434025764465,
  0.5364089608192444,
  0.6776167750358582,
  0.5689002275466919,
  0.6323218941688538,
  0.4941909611225128,
  0.6641713380813599,
  0.554228663444519,
  0.6021980047225952,
  0.6427145600318909,
  0.5987027883529663,
  0.599273145198822,
  0.5930231809616089,
  0.597343921661377,
  0.5315658450126648,
  0.5899679064750671,
  0.6405826210975647,
  0.5401877760887146,
  0.5037523508071899,
  0.5743625164031982,
  0.5797250270843506,
  0.5852159261703491,
  0.5745620131492615,
  0.531157910823822,
  0.49666881561279297,
  0.5582928657531738,
  0.6647825837135315,
  0.5930289626121521,
  0.5623475909233093,
  0.6347811222076416,
  0.5677080154418945,
  0.6170021295547485,
  0.6037501692771912,
  0.5434162020683289,
  0.5515047907829285,
  0.6771818995475769,
  0.65170222520828

In [0]:
font_config = {
    'family': 'serif',
    'weight': 'normal',
    'color': 'white',
    'size': 20
}

axe_font = {
    'family': 'serif',
    'weight': 'normal',
    'color': 'white',
    'size': 10
}

epoch = len(train_accu)
epochs = [x for x in range(1, epoch + 1)]

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

axes[0].set_xlabel('Epochs', fontdict=font_config)
axes[0].set_ylabel('Accuracy', fontdict=font_config)
axes[0].tick_params(axis='both', labelcolor='white', color='white')
axes[0].plot(epochs, train_accu, label='Train')
axes[0].plot(epochs, val_accu, label='Validation')
axes[0].legend()

axes[1].set_xlabel('Epochs', fontdict=font_config)
axes[1].set_ylabel('Loss', fontdict=font_config)
axes[1].tick_params(axis='both', labelcolor='white', color='white')
axes[1].plot(epochs, train_loss, label='Train')
axes[1].plot(epochs, val_loss, label='Validation')
axes[1].legend()

plt.show()